## Heuristics applied to pairs

In [1]:
from bertopic import BERTopic
from scipy.cluster import hierarchy as sch
import pandas as pd
import logging
from sklearn.metrics.pairwise import cosine_distances
from tqdm import tqdm
import numpy as np
from umap import UMAP
from hdbscan import HDBSCAN
from sentence_transformers import SentenceTransformer
import re
import sys
sys.path.append('../../')
from utility.utility_functions import *

In [3]:
df_change = pd.read_parquet('../../data/processed/intents/change.parquet', engine='pyarrow')

## Meeting

In [4]:
df_change_counts = create_count_df(df_change)
df_change_counts

,action_object_pairs,Count
0,send_e,1635
1,send_mail,1384
2,receive_e,1248
3,make_changes,1125
4,receive_mail,931
...,...,...
15035,send_myself,1
15036,provide_introductions,1
15037,receive_relief,1
15038,include_potential,1


### Inspect Pairs and create groups

In [5]:
df_change_counts.head(20)

,action_object_pairs,Count
0,send_e,1635
1,send_mail,1384
2,receive_e,1248
3,make_changes,1125
4,receive_mail,931
5,get_e,713
6,get_mail,678
7,change_address,533
8,make_change,389
9,give_update,383


Based on the Inspection, the following groups will be created:



reschedule_meeting

schedule_meeting, arrange_meeting, schedule_meetings, set_meetings, set_meeting, setup_meeting, set_appointment, have_meeting, have_discussion, have_discussions, have_meetings, have_appointment, have_conversation, hold_meeting,

 attend_conference, attend_meeting, attend_meetings


review_content

In [28]:
change = ['make_changes']

In [29]:
df_make_changes =  df_change[df_change['action_object_pairs'].apply(lambda pairs: any(item.lower() in change for item in pairs))]

In [34]:
df_make_changes.iloc[3]['target']

'Please use this form to make any changes applicable to your 401k deduction for next year and return it to H.R. by January 5 , 2001 .'

In [56]:
# Define the words or phrases to filter
#list_a = {'need','please'}
list_b = {'make', }
list_c = {'change', 'changes', }

# Combine List A and List B using OR, then combine with List C using AND
#pattern_a = '|'.join(rf'\b{word}\b' for word in list_a)  
pattern_b = '|'.join(rf'\b{word}\b' for word in list_b)   
pattern_c = '|'.join(rf'\b{word}\b' for word in list_c)   

# Apply the filter
df_make_changes_filtered = df_make_changes[
    #df_make_changes['target'].str.contains(pattern_a, case=False, na=False) &     
    df_make_changes['target'].str.contains(pattern_b, case=False, na=False) &    
    df_make_changes['target'].str.contains(pattern_c, case=False, na=False) 
]

df_non_matching = df_make_changes[~(
    #df_make_changes['target'].str.contains(pattern_a, case=False, na=False) &
    df_make_changes['target'].str.contains(pattern_b, case=False, na=False) &
    df_make_changes['target'].str.contains(pattern_c, case=False, na=False)
)]

In [61]:
df_make_changes_filtered.shape

(833, 19)

In [58]:
df_make_changes_filtered.iloc[11]['target']

'3/1 - 3/2 - write SFS and complete external review \r\n 3/5 - 3/9 - Admin tool code changes - here we may need to make minor \r\n\t changes to the EMAS code like URLencode the messages \r\n\t exchanged with server .'

In [59]:
index = 3
df_non_matching.iloc[index]['target']

'Solution : We have made some changes to the Exchange servers that will help improve reliability .'

In [60]:
# Create dataset
pd.DataFrame({
    'text': df_make_changes_filtered['target'],         
    'label': 'MakeChanges'   
}).to_csv("./changes.tsv",sep='\t',index=False)